In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [4]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    y_logits=np.array(y_logits)
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('sngan')):
    os.mkdir('saved_models_{}'.format('sngan'))
f = open('saved_models_{}/log_collapse1.txt'.format('sngan'), mode='w')
import torch.utils.data as Data
import cv2
x = []
y = np.zeros((31, 1), dtype=np.int)
y = list(y)
for i in range(31):
   y[i] = []
# for Generative Adversarial Networks
# Ref:
# - https: // arxiv.org / abs / 1802.05957
# - https: // github.com / pfnet - research / sngan_projection / tree / master / source
import keras
from keras.datasets import cifar10
import os
from scipy import misc
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose, Conv2D
from keras.optimizers import Adam
import os
from keras.layers.convolutional import _Conv
from keras.legacy import interfaces
from keras.engine import InputSpec
import keras.backend.tensorflow_backend as KTF
import matplotlib.pyplot as plt
from scipy import misc
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    KTF.set_session(tf.Session(config=config))


def predict_images(file_name, generator, noise_size, n = 10, size = 32):

    image = generator.predict(np.random.normal(size=(n*n, ) + noise_size))

    image = np.reshape(image, (n, n, size, size, 3))
    image = np.transpose(image, (0, 2, 1, 3, 4))
    image = np.reshape(image, (n*size, n*size, 3))

    image = 255 * (image + 1) / 2
    image = image.astype("uint8")
    misc.imsave(file_name, image)
def build_generator(input_shape):
    model = Sequential()

    model.add(Conv2DTranspose(512,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(3,(3,3),padding="same",activation="tanh"))
    model.summary()
    return model


def build_discriminator(input_shape):
    model = Sequential()

    model.add(SNConv2D(64,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(512,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(1,(3,3),padding="same"))
    model.add(GlobalAveragePooling2D())
    model.summary()

    return model

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    d_loss = K.mean(K.maximum(0., 1 - pred_real)) + K.mean(K.maximum(0., 1 + pred_fake))
    g_loss = -K.mean(pred_fake)

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss], d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_training_updates)

    return d_train,g_train

class SNConv2D(_Conv):
    @interfaces.legacy_conv2d_support
    def __init__(self, filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        super(SNConv2D, self).__init__(
            rank=2,
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)

        self.input_spec = InputSpec(ndim=4)
        self.Ip = 1
        self.u = self.add_weight(
            name='W_u',
            shape=(1,filters),
            initializer='random_uniform',
            trainable=False
        )

    def call(self, inputs):
        outputs = K.conv2d(
            inputs,
            self.W_bar(),
            strides=self.strides,
            padding=self.padding,
            data_format=self.data_format,
            dilation_rate=self.dilation_rate)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs


    def get_config(self):
        config = super(SNConv2D, self).get_config()
        config.pop('rank')
        return config

    def W_bar(self):
        # Spectrally Normalized Weight
        W_mat = K.permute_dimensions(self.kernel, (3, 2, 0, 1)) # (h, w, i, o) => (o, i, h, w)
        W_mat = K.reshape(W_mat,[K.shape(W_mat)[0], -1]) # (o, i * h * w)

        if not self.Ip >= 1:
            raise ValueError("The number of power iterations should be positive integer")

        _u = self.u
        _v = None

        for _ in range(self.Ip):
            _v = _l2normalize(K.dot(_u, W_mat))
            _u = _l2normalize(K.dot(_v, K.transpose(W_mat)))

        sigma = K.sum(K.dot(_u,W_mat)*_v)

        K.update(self.u,K.in_train_phase(_u, self.u))
        return self.kernel / sigma

def _l2normalize(x):
    return x / K.sqrt(K.sum(K.square(x)) + K.epsilon())
set_gpu_config("0",0.5)

epochs = 50
image_size = (32,32,3)
noise_size = (2,2,32)
batch_size = 64
sample_size=10
size=32
sample_interval=200
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_of_data = x_train.shape[0]
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = (x_train/255)*2-1
x_test = (x_test/255)*2-1
y_train = keras.utils.to_categorical(y_train,10)
# y_test = keras.utils.to_categorical(y_test,10)

generator = build_generator(noise_size)
discriminator = build_discriminator(image_size)
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

nb_batches = int(x_train.shape[0] / batch_size)
global_step = 0
steps = []
values = []
for epoch in range(epochs):
    for index in range(nb_batches):
        global_step += 1
        real_images = x_train[index * batch_size:(index + 1) * batch_size]
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        print("[{0}/{1}] [{2}_{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(epoch, epochs, epoch, global_step, d_loss,
                                                                              g_loss))
        sample_num = 5000
        # If at save interval => save generated image samples
        if global_step % sample_interval == 0:
            x.append(epoch)
            gen_imgs = generator.predict(np.random.normal(size=(sample_size,) + noise_size))
            # gen_imgs = np.reshape(gen_imgs, (sample_size, sample_size, size, size, 3))
            # gen_imgs = np.transpose(gen_imgs, (0, 2, 1, 3, 4))
            # gen_imgs = np.reshape(gen_imgs, (sample_size * size, sample_size * size, 3))

            gen_imgs = 255 * (gen_imgs + 1) / 2
            
            if global_step % sample_interval == 0:
                labels = np.squeeze(y_test[:sample_num])
                labels = np.squeeze(labels)
                dis_real = cal_distance_image_real(x_test[:sample_num], labels)
                dis_fake = cal_distance_image_fake(gen_imgs)
                dis_cha = dis_real - dis_fake
                print('##############')
                # print(dis_real)
                # print(dis_fake)
                print(dis_cha)
                print('##########')
                f.writelines('\n')
                f.writelines('step:' + str(global_step))
                f.writelines('\n')
                f.writelines('紧度')
                f.writelines('\n')
                f.writelines(' %.8f ' % (i) for i in dis_cha)
                f.writelines('\n')

Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_9 (Conv2DTr (None, 4, 4, 512)         147968    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 8, 8, 256)         1179904   
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_11 (Conv2DT (None, 16, 16, 128)       295040    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_12 (Conv2DT (None, 32, 32, 64)        73792     
__________

[0/50] [0_113] d_loss: 0.5225, g_loss: 3.838
[0/50] [0_114] d_loss: 0.005295, g_loss: 2.644
[0/50] [0_115] d_loss: 0.9596, g_loss: 3.193
[0/50] [0_116] d_loss: 0.1854, g_loss: 2.374
[0/50] [0_117] d_loss: 0.1034, g_loss: 1.443
[0/50] [0_118] d_loss: 0.8458, g_loss: 2.356
[0/50] [0_119] d_loss: 0.1224, g_loss: 1.765
[0/50] [0_120] d_loss: 0.1418, g_loss: 1.418
[0/50] [0_121] d_loss: 0.3501, g_loss: 4.202
[0/50] [0_122] d_loss: 0.4973, g_loss: 1.393
[0/50] [0_123] d_loss: 0.1089, g_loss: 1.644
[0/50] [0_124] d_loss: 0.2271, g_loss: 1.158
[0/50] [0_125] d_loss: 0.4005, g_loss: 5.592
[0/50] [0_126] d_loss: 0.6658, g_loss: 0.4004
[0/50] [0_127] d_loss: 1.029, g_loss: 1.204
[0/50] [0_128] d_loss: 0.2044, g_loss: 1.202
[0/50] [0_129] d_loss: 0.1802, g_loss: 4.27
[0/50] [0_130] d_loss: 0.3669, g_loss: 0.5545
[0/50] [0_131] d_loss: 1.408, g_loss: 1.848
[0/50] [0_132] d_loss: 1.063, g_loss: 0.7227
[0/50] [0_133] d_loss: 0.5544, g_loss: 1.238
[0/50] [0_134] d_loss: 0.2967, g_loss: 0.653
[0/50] [0

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:57: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


##############
[1.118885   1.09672887 1.10657782 2.1030552  2.11611612 1.10923695
 2.10273311 1.012805   1.11042304 1.11991546]
##########
[0/50] [0_201] d_loss: 0.03545, g_loss: 2.85
[0/50] [0_202] d_loss: 0.04794, g_loss: 1.647
[0/50] [0_203] d_loss: 0.2025, g_loss: 6.349
[0/50] [0_204] d_loss: 0.6907, g_loss: 3.131
[0/50] [0_205] d_loss: 0.0392, g_loss: 2.64
[0/50] [0_206] d_loss: 0.0201, g_loss: 2.189
[0/50] [0_207] d_loss: 0.004697, g_loss: 1.546
[0/50] [0_208] d_loss: 0.3917, g_loss: 4.821
[0/50] [0_209] d_loss: 0.2782, g_loss: 2.659
[0/50] [0_210] d_loss: 0.08091, g_loss: 2.157
[0/50] [0_211] d_loss: 0.0815, g_loss: 1.49
[0/50] [0_212] d_loss: 0.2472, g_loss: 4.131
[0/50] [0_213] d_loss: 0.08254, g_loss: 3.041
[0/50] [0_214] d_loss: 0.0, g_loss: 1.7
[0/50] [0_215] d_loss: 1.217, g_loss: 2.971
[0/50] [0_216] d_loss: 0.1328, g_loss: 2.455
[0/50] [0_217] d_loss: 0.09543, g_loss: 1.583
[0/50] [0_218] d_loss: 0.6592, g_loss: 4.858
[0/50] [0_219] d_loss: 0.1635, g_loss: 3.317
[0/50] [

...

[49/50] [49_38296] d_loss: 0.3417, g_loss: 2.393
[49/50] [49_38297] d_loss: 0.3134, g_loss: 1.85
[49/50] [49_38298] d_loss: 0.6827, g_loss: 2.774
[49/50] [49_38299] d_loss: 0.3691, g_loss: 1.955
[49/50] [49_38300] d_loss: 0.4623, g_loss: 2.617
[49/50] [49_38301] d_loss: 0.4656, g_loss: 1.303
[49/50] [49_38302] d_loss: 0.7928, g_loss: 2.38
[49/50] [49_38303] d_loss: 0.8294, g_loss: 0.8341
[49/50] [49_38304] d_loss: 0.8179, g_loss: 1.409
[49/50] [49_38305] d_loss: 0.4292, g_loss: 1.301
[49/50] [49_38306] d_loss: 0.4312, g_loss: 1.512
[49/50] [49_38307] d_loss: 0.2497, g_loss: 2.499
[49/50] [49_38308] d_loss: 0.3748, g_loss: 2.121
[49/50] [49_38309] d_loss: 0.4823, g_loss: 2.555
[49/50] [49_38310] d_loss: 0.5687, g_loss: 1.128
[49/50] [49_38311] d_loss: 0.6721, g_loss: 1.96
[49/50] [49_38312] d_loss: 0.6896, g_loss: 1.56
[49/50] [49_38313] d_loss: 0.5558, g_loss: 2.177
[49/50] [49_38314] d_loss: 0.5872, g_loss: 2.722
[49/50] [49_38315] d_loss: 0.701, g_loss: 1.438
[49/50] [49_38316] d_los

[49/50] [49_38464] d_loss: 0.4773, g_loss: 1.382
[49/50] [49_38465] d_loss: 0.6246, g_loss: 2.6
[49/50] [49_38466] d_loss: 0.7332, g_loss: 1.034
[49/50] [49_38467] d_loss: 0.7715, g_loss: 2.057
[49/50] [49_38468] d_loss: 0.8399, g_loss: 1.06
[49/50] [49_38469] d_loss: 0.5076, g_loss: 2.019
[49/50] [49_38470] d_loss: 0.2444, g_loss: 1.969
[49/50] [49_38471] d_loss: 0.3555, g_loss: 2.479
[49/50] [49_38472] d_loss: 0.4365, g_loss: 1.866
[49/50] [49_38473] d_loss: 0.3812, g_loss: 2.158
[49/50] [49_38474] d_loss: 0.226, g_loss: 2.601
[49/50] [49_38475] d_loss: 0.4517, g_loss: 1.83
[49/50] [49_38476] d_loss: 0.4592, g_loss: 2.421
[49/50] [49_38477] d_loss: 0.7645, g_loss: 1.484
[49/50] [49_38478] d_loss: 0.5631, g_loss: 1.751
[49/50] [49_38479] d_loss: 0.5209, g_loss: 1.596
[49/50] [49_38480] d_loss: 0.5233, g_loss: 1.817
[49/50] [49_38481] d_loss: 0.4222, g_loss: 2.229
[49/50] [49_38482] d_loss: 0.2454, g_loss: 1.478
[49/50] [49_38483] d_loss: 0.6305, g_loss: 1.332
[49/50] [49_38484] d_loss

[49/50] [49_38633] d_loss: 0.4347, g_loss: 2.457
[49/50] [49_38634] d_loss: 0.3751, g_loss: 1.536
[49/50] [49_38635] d_loss: 0.4819, g_loss: 2.154
[49/50] [49_38636] d_loss: 0.3321, g_loss: 2.075
[49/50] [49_38637] d_loss: 0.2858, g_loss: 1.279
[49/50] [49_38638] d_loss: 0.5807, g_loss: 2.327
[49/50] [49_38639] d_loss: 0.4649, g_loss: 1.264
[49/50] [49_38640] d_loss: 0.4823, g_loss: 2.465
[49/50] [49_38641] d_loss: 0.4759, g_loss: 3.003
[49/50] [49_38642] d_loss: 0.2491, g_loss: 1.686
[49/50] [49_38643] d_loss: 0.3629, g_loss: 3.114
[49/50] [49_38644] d_loss: 0.7429, g_loss: 1.037
[49/50] [49_38645] d_loss: 0.6, g_loss: 2.839
[49/50] [49_38646] d_loss: 0.4902, g_loss: 1.619
[49/50] [49_38647] d_loss: 0.3461, g_loss: 2.138
[49/50] [49_38648] d_loss: 0.3704, g_loss: 2.675
[49/50] [49_38649] d_loss: 0.6014, g_loss: 1.615
[49/50] [49_38650] d_loss: 0.6084, g_loss: 2.214
[49/50] [49_38651] d_loss: 0.791, g_loss: 1.465
[49/50] [49_38652] d_loss: 0.4987, g_loss: 2.305
[49/50] [49_38653] d_los

##############
[0.93102738 1.09262168 1.10991144 1.06534578 2.11560126 2.10903264
 2.10986634 1.11543146 1.1082014  1.12178249]
##########
[49/50] [49_38801] d_loss: 0.5822, g_loss: 2.055
[49/50] [49_38802] d_loss: 0.3659, g_loss: 1.301
[49/50] [49_38803] d_loss: 0.4677, g_loss: 1.262
[49/50] [49_38804] d_loss: 0.6003, g_loss: 1.798
[49/50] [49_38805] d_loss: 0.4997, g_loss: 2.418
[49/50] [49_38806] d_loss: 0.5164, g_loss: 1.496
[49/50] [49_38807] d_loss: 0.3919, g_loss: 1.941
[49/50] [49_38808] d_loss: 0.3704, g_loss: 1.707
[49/50] [49_38809] d_loss: 0.2668, g_loss: 2.676
[49/50] [49_38810] d_loss: 0.587, g_loss: 1.699
[49/50] [49_38811] d_loss: 0.3844, g_loss: 1.233
[49/50] [49_38812] d_loss: 0.6415, g_loss: 4.058
[49/50] [49_38813] d_loss: 0.7174, g_loss: 0.943
[49/50] [49_38814] d_loss: 0.6765, g_loss: 1.964
[49/50] [49_38815] d_loss: 0.6227, g_loss: 2.346
[49/50] [49_38816] d_loss: 0.2858, g_loss: 1.392
[49/50] [49_38817] d_loss: 0.2956, g_loss: 2.618
[49/50] [49_38818] d_loss: 0.

[49/50] [49_38969] d_loss: 0.2728, g_loss: 1.707
[49/50] [49_38970] d_loss: 0.5065, g_loss: 2.795
[49/50] [49_38971] d_loss: 0.4322, g_loss: 2.356
[49/50] [49_38972] d_loss: 0.4471, g_loss: 2.23
[49/50] [49_38973] d_loss: 0.4281, g_loss: 2.465
[49/50] [49_38974] d_loss: 0.2284, g_loss: 1.859
[49/50] [49_38975] d_loss: 0.6123, g_loss: 1.863
[49/50] [49_38976] d_loss: 0.4253, g_loss: 1.982
[49/50] [49_38977] d_loss: 0.3956, g_loss: 2.431
[49/50] [49_38978] d_loss: 0.296, g_loss: 2.247
[49/50] [49_38979] d_loss: 0.2901, g_loss: 2.014
[49/50] [49_38980] d_loss: 0.4118, g_loss: 1.881
[49/50] [49_38981] d_loss: 0.4751, g_loss: 3.085
[49/50] [49_38982] d_loss: 0.6167, g_loss: 0.8846
[49/50] [49_38983] d_loss: 0.7504, g_loss: 2.316
[49/50] [49_38984] d_loss: 0.6415, g_loss: 1.084
[49/50] [49_38985] d_loss: 0.6969, g_loss: 2.545
[49/50] [49_38986] d_loss: 0.4774, g_loss: 1.981
[49/50] [49_38987] d_loss: 0.4089, g_loss: 2.064
[49/50] [49_38988] d_loss: 0.4982, g_loss: 2.896
[49/50] [49_38989] d_